In [156]:
from src.data_ingestion.db.postgres_db import PostgresConfig
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine
import pandas as pd
import datetime

load_dotenv()

db_uri = PostgresConfig(
    host=os.environ["SB_DDBB_HOST"],
    port=os.environ["SB_DDBB_PORT"],
    database=os.environ["SB_DDBB_DATABASE"],
    user=os.environ["SB_DDBB_USER"],
    password=os.environ["SB_DDBB_PWD"]
).get_connection_string()

engine = create_engine(db_uri)

df_current_portfolio = pd.read_sql("SELECT * FROM portfolio WHERE created_at = (SELECT max(created_at) FROM portfolio)", engine)

df_prices = pd.read_sql("SELECT * FROM stock_price_daily WHERE date >= '2024-06-01'", engine)

In [159]:
df_current_portfolio["date"] = df_current_portfolio["created_at"].dt.date.astype("datetime64[ns]") - datetime.timedelta(days=60)
df_current_portfolio

,asset_name,asset_type,weight,created_at,asset_short_name,portfolio_trader,date
0,IBM,stock,0.150,2024-08-05 12:08:41.332903+00:00,IBM,QuantTraderNaiveAdal,2024-06-06
1,Palantir,stock,0.200,2024-08-05 12:08:41.332903+00:00,PLTR,QuantTraderNaiveAdal,2024-06-06
2,Ford Motor Company,stock,0.050,2024-08-05 12:08:41.332903+00:00,F,QuantTraderNaiveAdal,2024-06-06
3,Oatly Group AB,stock,0.050,2024-08-05 12:08:41.332903+00:00,OTLY,QuantTraderNaiveAdal,2024-06-06
4,"Corteva, Inc.",stock,0.050,2024-08-05 12:08:41.332903+00:00,CTVA,QuantTraderNaiveAdal,2024-06-06
5,"VeriSign, Inc.",stock,0.050,2024-08-05 12:08:41.332903+00:00,VRSN,QuantTraderNaiveAdal,2024-06-06
6,Adyen N.V.,stock,0.050,2024-08-05 12:08:41.332903+00:00,ADYEN,QuantTraderNaiveAdal,2024-06-06
7,"Beyond Meat, Inc.",stock,0.050,2024-08-05 12:08:41.332903+00:00,BYND,QuantTraderNaiveAdal,2024-06-06
8,"Altria Group, Inc.",stock,0.030,2024-08-05 12:08:41.332903+00:00,MO,QuantTraderNaiveAdal,2024-06-06
9,The Mosaic Company,stock,0.035,2024-08-05 12:08:41.332903+00:00,MOS,QuantTraderNaiveAdal,2024-06-06


In [160]:
def expand_df_dates(df, date_column, start_date, end_date, inclusive):
    date_range = pd.date_range(start=start_date, end=end_date, inclusive=inclusive)
    df_date= pd.DataFrame(date_range, columns=[date_column])
    
    return pd.merge(
        df_date,
        df,
        on=date_column,
        how="left"
    )

def generate_portfolio_evolution(df, start_date, end_date, inclusive):
    df_portfolio_expanded = expand_df_dates(df, "date", start_date, end_date, inclusive)

    analized_date = None
    past_portfolio =  pd.DataFrame()
    final_portfolio = pd.DataFrame()
    for row in df_portfolio_expanded.itertuples():
        if row.date != analized_date:
            df_filtered = df_portfolio_expanded[df_portfolio_expanded.date == row.date]
            analized_date = row.date

            if df_filtered.shape[0] > 1:
                past_portfolio = df_filtered
                final_portfolio = pd.concat(
                    [final_portfolio,
                    df_filtered]
                )
            else:
                past_portfolio["date"] = row.date
                final_portfolio = pd.concat(
                    [final_portfolio,
                    past_portfolio]
                )
    return final_portfolio


In [166]:
df_portfolio = generate_portfolio_evolution(df_current_portfolio, "2024-06-01", "2024-07-27", "left")
df_portfolio.drop(columns=["created_at"], inplace=True)

/var/folders/79/tn_k0n6s6wsdt_7jnfyh9rm40000gn/T/ipykernel_1819/1060967528.py:30: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [167]:
df_prices["asset_short_name"] = df_prices["ticker"]
df_prices["date"] = df_prices["date"].astype("datetime64[ns]")


In [168]:
df_merged = pd.merge(df_portfolio, df_prices, on=['asset_short_name','date'], how='left')

In [189]:
#df_merged[df_merged["asset_short_name"]=='IBM']
df_merged["value"] = df_merged["weight"] * df_merged["close"]

agg_portfolio_value = df_merged.groupby(["date"], as_index=False).agg({"value": "sum"})
agg_portfolio_value = agg_portfolio_value[agg_portfolio_value["value"]>0]
agg_portfolio_value["value_perc_vs_init"] = (agg_portfolio_value["value"] - agg_portfolio_value["value"][0]) / agg_portfolio_value["value"][0] * 100
agg_portfolio_value.head(4)

,date,value,value_perc_vs_init
0,2024-06-06,55.413674,0.000000
1,2024-06-07,55.571560,0.284922
4,2024-06-10,55.817819,0.729323
5,2024-06-11,55.110823,-0.546527


In [190]:
agg_assset_value = df_merged.groupby(["date", "asset_name"], as_index=False).agg({"value": "sum"})
agg_assset_value = agg_assset_value[agg_assset_value["value"]>0]

In [191]:
import plotly.express as p


p.line(agg_portfolio_value, x="date", y="value", title="Dystopic Portfolio Evolution")

In [296]:
import plotly.graph_objs as go

negative_traces_df = expand_df_dates(
    agg_portfolio_value[agg_portfolio_value["value_perc_vs_init"] < 0],
    "date",
    agg_portfolio_value.date.tolist()[0],
    agg_portfolio_value.date.tolist()[-1],
    "both",
)

fig2 = p.line(
    agg_portfolio_value, x="date", y="value_perc_vs_init", title="Dystopic Portfolio Performance Evolution")
fig2.add_trace(
    go.Scatter(x = negative_traces_df.date, y =negative_traces_df.value_perc_vs_init, connectgaps=True, name="negative", showlegend=False),
)
fig2.update_traces(selector=lambda x: x["name"] != "negative",line_color='#147852')
fig2.update_legends(overwrite=True)
# fig2.update_traces(connectgaps=True)

In [183]:
p.line(
    agg_assset_value,
    x="date",
    y="value",
    color="asset_name",
    title="Assets Evolution",
    facet_col="asset_name",
)